In [ ]:
import ee

import pandas as pd
import re
import matplotlib.pyplot as plt
%matplotlib inline

from datetime import date
from datetime import timedelta
import zipfile
import os
import requests

import numpy as np

import geopy.distance
!pip install geopandas
import geopandas  as gpd
from shapely.geometry import Point, LineString, Polygon


# Import specific function 'from_epsg' from fiona module
from fiona.crs import from_epsg

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 1.0 MB 4.7 MB/s 
     |████████████████████████████████| 16.7 MB 317 kB/s 
     |████████████████████████████████| 6.3 MB 43.0 MB/s 


In [ ]:
# connect to google drive
from google.colab import drive  

# drive roots
DRIVE_MOUNT = '/content/drive'
drive.mount(DRIVE_MOUNT, force_remount=True)

# set up project directories
PROJECT_DIR = DRIVE_MOUNT + "/My Drive/Volcano_datasets/"
TMP_DIR = PROJECT_DIR + "TempDIR/"
targets = pd.read_excel(PROJECT_DIR + "Sample Eruptions.xlsx")

OUTPUT_SHAPE_DIR = PROJECT_DIR+"output_damage_shapes/"

Mounted at /content/drive


In [ ]:
distances = [1,5,100,500,1000]
newdata = gpd.GeoDataFrame()
id = 0
for ivol in range(targets.shape[0]):
  volc = targets.iloc[ivol]
  vloc = Point( volc["Longitude"],volc["Latitude"])
  v_name = re.sub(r"[,.]","",re.sub(r"\s+", "_",volc["Volcano Name"]))
  erupt_start_date = date(volc["Start Year"].astype(int),
                            volc["Start Month"].astype(int),
                            volc["Start Day"].astype(int))
  
  for dist in distances:
    pts = []
    for bearing in range(0,350,10):
      pt =  geopy.distance.distance(kilometers=(dist - ((dist/2) * np.random.rand())))\
                   .destination((volc["Latitude"], volc["Longitude"]), bearing=bearing)[:2]
      pts.append([pt[1],pt[0]])
    poly = Polygon(pts)
    newdata.loc[id,'geometry'] = poly
    newdata.loc[id,"Volc_Name"] = v_name
    newdata.loc[id,"Volc_Num"] = volc["Volcano Number"]
    newdata.loc[id,"Erupt_Num"] = volc["Eruption Number"]
    newdata.loc[id,"Est_Date"] = erupt_start_date.isoformat()
    newdata.loc[id,"layer"] = dist
    id = id +1
    
print(newdata)
newdata.set_crs('EPSG:3857')
output_file = f'{OUTPUT_SHAPE_DIR}/all_data.shp'
newdata.to_file(output_file)

                                              geometry         Volc_Name  \
0    POLYGON ((130.21700 30.44909, 130.21869 30.451...  Kuchinoerabujima   
1    POLYGON ((130.21700 30.48262, 130.22451 30.479...  Kuchinoerabujima   
2    POLYGON ((130.21700 31.00062, 130.32443 30.968...  Kuchinoerabujima   
3    POLYGON ((130.21700 33.50320, 131.13979 34.764...  Kuchinoerabujima   
4    POLYGON ((130.21700 36.38784, 131.28065 35.386...  Kuchinoerabujima   
..                                                 ...               ...   
130  POLYGON ((157.97900 -8.98520, 157.97982 -8.986...           Kavachi   
131  POLYGON ((157.97900 -8.96580, 157.98307 -8.968...           Kavachi   
132  POLYGON ((157.97900 -8.27949, 158.06318 -8.515...           Kavachi   
133  POLYGON ((157.97900 -6.35922, 158.42619 -6.453...           Kavachi   
134  POLYGON ((157.97900 -4.11519, 159.35471 -1.112...           Kavachi   

     Volc_Num  Erupt_Num    Est_Date   layer  
0    282050.0    22345.0  2020-01-11    